#**LINK TO THE PRESENTATION VIDEO**

https://drive.google.com/file/d/1f5XzpDtxquyzzaCCq_6Lr0xnUfNTFw_l/view?usp=sharing

# **Notebook setup**

In [ ]:
!apt install clang-10

Reading package lists... Done
Building dependency tree       
Reading state information... Done
clang-10 is already the newest version (1:10.0.0-4ubuntu1~18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


**Download the code**

In [ ]:
!git clone https://github.com/google/benchmark.git
!git clone https://github.com/google/googletest.git benchmark/googletest

Cloning into 'benchmark'...
remote: Enumerating objects: 6571, done.
remote: Counting objects: 100% (827/827), done.
remote: Compressing objects: 100% (454/454), done.
remote: Total 6571 (delta 476), reused 590 (delta 327), pack-reused 5744
Receiving objects: 100% (6571/6571), 2.23 MiB | 8.65 MiB/s, done.
Resolving deltas: 100% (4245/4245), done.
Cloning into 'benchmark/googletest'...
remote: Enumerating objects: 23334, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 23334 (delta 120), reused 141 (delta 75), pack-reused 23100
Receiving objects: 100% (23334/23334), 9.56 MiB | 17.74 MiB/s, done.
Resolving deltas: 100% (17185/17185), done.


**Organize the code and install**

In [ ]:
!rm -rf benchmark/build
!cmake -E make_directory "benchmark/build"
!cmake -E chdir "benchmark/build" cmake -DCMAKE_BUILD_TYPE=Release ..
!cmake --build "benchmark/build" --config Release --target install

-- The CXX compiler identification is GNU 7.5.0
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Failed to find LLVM FileCheck
-- Found Git: /usr/bin/git (found version "2.17.1") 
-- git version: v1.6.0-10-g7fad964a normalized to 1.6.0.10
-- Version: 1.6.0.10
-- Performing Test HAVE_CXX_FLAG_STD_CXX11
-- Performing Test HAVE_CXX_FLAG_STD_CXX11 - Success
-- Performing Test HAVE_CXX_FLAG_WALL
-- Performing Test HAVE_CXX_FLAG_WALL - Success
-- Performing Test HAVE_CXX_FLAG_WEXTRA
-- Performing Test HAVE_CXX_FLAG_WEXTRA - Success
-- Performing Test HAVE_CXX_FLAG_WSHADOW
-- Performing Test HAVE_CXX_FLAG_WSHADOW - Success
-- Performing Test HAVE_CXX_FLAG_WERROR
-- Performing Test HAVE_CXX_FLAG_WERROR - Success
-- Performing Test HAVE_CXX_FLAG_PEDANTIC
-- Performing Test HAVE_CXX_FLAG_

In [ ]:
!mkdir original_impl
!mkdir array_impl

# **Profiling the Karger and the Karger&Stein mincut algorithm**
# *Original Implementation*

In [ ]:
%%writefile original_impl/karger.hpp
#pragma once

#include <algorithm>
#include <cmath>
#include <random>
#include <stack>
#include <vector>


/* The following function returns an engine for random number generation */
auto& prng_engine() {
    thread_local static std::mt19937 engine{std::random_device{}()};
    return engine;
} 

/* template <typename node_t> is added to tell at the struct below to use a "generic type" that will be specified after. 
Instead of node_t we could have used whatever placeholder */

template <typename node_t>
struct Edge { node_t tail, head; };

/* Represents a graph with n vertices as a collection of edges whose vertices are indexed between 0
and n - 1. */

template <typename node_t>
struct EdgesVectorGraph
{
    node_t n; // number of vertices
    std::vector<Edge<node_t>> edges;
};


/* An Union-Find data structure (https://fr.wikipedia.org/wiki/Union-Find) is a data structure that
stores a partition of a set into disjoint subsets. Tree height is controlled using union by size.
Use path compression technique. */


template <typename T>
struct UnionFind
{
    struct Subset { T id, size; Subset(T _id, T _size) : id(_id), size(_size) {}};
    std::vector<Subset> subsets;
    T nb_subsets;

    UnionFind(T n) : nb_subsets{n} { 
        subsets.reserve(n);
        for (T i = 0; i < n; ++i)
            subsets.emplace_back(i, 1);
    }

    auto find(T x) {
        auto root = x;
        while (root != subsets[root].id) { root = subsets[root].id; }
        while (subsets[x].id != root) { auto id = subsets[x].id; subsets[x].id = root; x = id; } /*performing path compression each time*/
        return root;
    }

    void merge(T x, T y) {
        auto const i = find(x); auto const j = find(y);
        if (i == j) return;
        if (subsets[i].size < subsets[j].size) { subsets[i].id = j; subsets[j].size += subsets[i].size; }
        else                                   { subsets[j].id = i; subsets[i].size += subsets[j].size; }
        --nb_subsets;
    }

    bool connected(T x, T y) { return find(x) == find(y); }
};


/* A data structure representing a cut of a graph. */
template <typename node_t>
struct GraphCut
{
    std::size_t cut_size;
    UnionFind<node_t> uf; // used to identify the two partitions of nodes
    GraphCut(std::size_t _cut_size, UnionFind<node_t> _uf) : cut_size(_cut_size), uf(_uf) {}

    bool operator<(GraphCut const& other) const { return cut_size < other.cut_size; }

    auto get_partitions() const {
        std::vector<node_t> P, Q;
        P.reserve(uf.subsets[0].size); Q.reserve(std::size(uf.subsets) - std::size(P));
        for (node_t i = 0; i < std::size(uf.subsets); ++i)
            uf.subsets[i].id == uf.subsets[0].id ? P.push_back(i) : Q.push_back(i);
        return std::array{P, Q};
    }
};


/* Karger's contraction algorithm in O(n + mα(n)) using an Union-Find data structure to keep track
of merged vertices. The graph is assumed to be connected and nodes indexed between 0 and n-1. Repeat
this function C(n,2)*log(n) = n*(n-1)/2*log(n) for high probability of obtaining the minimum global
cut. The graph isn't per se modifed, only its vector of edges is shuffled. */
template <typename node_t>
GraphCut<node_t> karger_union_find(EdgesVectorGraph<node_t>& graph)
{
    auto& mt = prng_engine();
    UnionFind uf{graph.n};
    auto start = begin(graph.edges);
    for (int m = size(graph.edges) - 1; uf.nb_subsets != 2; ++start, --m) {
        std::iter_swap(start, start + std::uniform_int_distribution<>{0, m}(mt));
        uf.merge(start->tail, start->head);
    }
    return {(std::size_t) std::count_if(start, end(graph.edges), [&](auto e)
        { return !uf.connected(e.tail, e.head); }), std::move(uf)};
}


/* Kargen-Stein's contraction recursive algorithm. Instead of using a straighforward recursion, we
keep the intermediate graphs to contract in a stack. Repeat this function log²(n) for high probabili
-ty of obtaining the minimum global cut. */
template <typename node_t>
auto karger_stein_union_find(EdgesVectorGraph<node_t> const& input_graph)
{
    /* A data structure to hold an intermediate contracted graph state. The Union-Find structure
    is used to keep track of the merged nodes. */ 
    struct ContractedGraph : EdgesVectorGraph<node_t> { UnionFind<node_t> uf; };

    /* Contracts the given graph until it has nb_vertices vertices. The graph isn't per se modifed,
    only its vector of edges is shuffled. */
    auto contract = [&mt = prng_engine()](ContractedGraph& graph, node_t nb_vertices) {   
        UnionFind uf{graph.uf};
        auto start = begin(graph.edges);
        for (int m = size(graph.edges) - 1; uf.nb_subsets != nb_vertices; ++start, --m) {
            std::iter_swap(start, start + std::uniform_int_distribution<>{0, m}(mt));
            uf.merge(start->tail, start->head);
        }
        decltype(graph.edges) edges;
        edges.reserve(end(graph.edges) - start);
        std::copy_if(start, end(graph.edges), std::back_inserter(edges),
            [&](auto e){ return !uf.connected(e.tail, e.head); }); // remove self-loops
        return ContractedGraph{nb_vertices, std::move(edges), std::move(uf)};
    };

    /* Returns the cut represented by an intermediate contracted graph. The given graph is supposed
    to have no self-loops and to have two nodes (components). */
    auto cut = [](ContractedGraph&& graph){ return GraphCut{std::size(graph.edges), graph.uf}; };
    
    double INV_SQRT_2 = 1.0 / std::sqrt(2);
    GraphCut<node_t> best_minimum_cut{input_graph.n, {{}}};
    std::stack<ContractedGraph, std::vector<ContractedGraph>> graphs;
    graphs.push({input_graph.n, input_graph.edges, {input_graph.n}});

    while (!graphs.empty()) // algorithm's main loop
    {
        auto graph = graphs.top();
        graphs.pop();

        if (graph.n <= 6) {
            best_minimum_cut = std::min(best_minimum_cut, cut(contract(graph, 2)));
        } else {
            node_t t = 1 + std::ceil(graph.n * INV_SQRT_2);
            graphs.push(contract(graph, t));
            graphs.push(contract(graph, t));
        }
    }

    return best_minimum_cut;
}

Overwriting original_impl/karger.hpp


In [ ]:
%%writefile original_impl/instance_reader.hpp
#pragma once

#include <fstream>
#include <sstream>
#include <string>
#include <vector>
#include "karger.hpp"


template <typename node_t>
EdgesVectorGraph<node_t> read_col_instance(std::string_view file) {
    EdgesVectorGraph<node_t> graph;
    std::ifstream instance;
    instance.open(file.data());
    if (!instance) throw std::runtime_error("Such instance doesn't exist.");
    for (std::string line; std::getline(instance, line);)
    {
        std::istringstream input(line);
        switch(line[0]) {
            case 'p':
                input.ignore(6);
                std::size_t m;
                input >> graph.n >> m;
                graph.edges.reserve(m);
                break;
            case 'e':
                input.ignore(2);
                Edge<node_t> edge;
                input >> edge.tail >> edge.head;
                --edge.tail; --edge.head; // instance files starting vertex is 1
                graph.edges.push_back(std::move(edge));
                break;
            default:
                break;
        }
    }
    return graph;
}

Overwriting original_impl/instance_reader.hpp


In [ ]:
%%writefile original_impl/MinCutBM.cpp
#include <algorithm>
#include <iostream>
#include <fstream>
#include <stdlib.h>
#include <cstdint>
#include <functional>
#include <array>

#include <benchmark/benchmark.h>

#include "karger.hpp"
#include "instance_reader.hpp"
static void KargerBM(benchmark::State& state)
{

    using node_t = std::uint32_t;       
            
    for (auto _ : state){
              
      state.PauseTiming();

      int n= state.range(0);
      int m= state.range(1);

      std::string file_name= "/content/toys/"+std::to_string(n) + "_" + std::to_string(m)+".col";

      auto graph = read_col_instance<node_t>(file_name);

      struct MinimumCutAlgorithm {
        std::string name;
        std::function<GraphCut<node_t>(EdgesVectorGraph<node_t>&)> algorithm;
        std::size_t nb_repeat;
        auto operator()(EdgesVectorGraph<node_t>& graph) const { return algorithm(graph); }
      };

      MinimumCutAlgorithm algorithm = {"Karger",       karger_union_find<node_t>,       static_cast<std::size_t>(0.5 * graph.n * (graph.n - 1) * std::log(graph.n))};


      GraphCut<node_t> best_minimum_cut{graph.n, {{}}};

      state.ResumeTiming();

      for(std::size_t i = algorithm.nb_repeat; i; --i)
        best_minimum_cut = std::min(best_minimum_cut, algorithm(graph));      
    }
    state.SetComplexityN(state.range(0));
}

static void KargerSteinBM(benchmark::State& state)
{

    using node_t = std::uint32_t;       
            
    for (auto _ : state){
              
      state.PauseTiming();

      int n= state.range(0);
      int m= state.range(1);

      std::string file_name= "/content/toys/"+std::to_string(n) + "_" + std::to_string(m)+".col";

      auto graph = read_col_instance<node_t>(file_name);

      struct MinimumCutAlgorithm {
        std::string name;
        std::function<GraphCut<node_t>(EdgesVectorGraph<node_t>&)> algorithm;
        std::size_t nb_repeat;
        auto operator()(EdgesVectorGraph<node_t>& graph) const { return algorithm(graph); }
      };

      MinimumCutAlgorithm algorithm = {"Karger-Stein", karger_stein_union_find<node_t>, static_cast<std::size_t>(std::log(graph.n) * std::log(graph.n))};

      GraphCut<node_t> best_minimum_cut{graph.n, {{}}};

      state.ResumeTiming();

      for(std::size_t i = algorithm.nb_repeat; i; --i)
        best_minimum_cut = std::min(best_minimum_cut, algorithm(graph));       
    }
    state.SetComplexityN(state.range(0));
}

static void CustomArguments(benchmark::internal::Benchmark *b){    
    for(int i = 8; i<=256; i=i*2){
        for(int j = i-1; j <= i*(i-1)/2 ; j=j*2){
            b->Args({i, j});
        }
    }
}


BENCHMARK(KargerBM)
    ->Apply(CustomArguments)
    ->Complexity(benchmark::BigO::oNSquared);

BENCHMARK(KargerSteinBM)
    ->Apply(CustomArguments)
    ->Complexity(benchmark::BigO::oNLogN);


BENCHMARK_MAIN();

Overwriting original_impl/MinCutBM.cpp


# **Profiling the Karger and the Karger&Stein mincut algorithm**
# *Revisited*

In [ ]:
%%writefile array_impl/karger.hpp
#pragma once

#include <algorithm>
#include <cmath>
#include <random>
#include <stack>
#include <vector>


auto& prng_engine() {
    thread_local static std::mt19937 engine{std::random_device{}()};
    return engine;
} 


template <typename node_t>
struct Edge { node_t tail, head; };

/* Represents a graph with n vertices as a collection of edges whose vertices are indexed between 0
and n - 1. */
template <typename node_t>
struct EdgesVectorGraph
{   
    
    node_t  n; // number of vertices
    node_t  m; 
    Edge<node_t>* edges = NULL;
    EdgesVectorGraph (node_t _n, node_t _m, Edge<node_t>* _edges) {
        n = _n;
        m = _m;
        edges = _edges;
    };
    
    
};

/* An Union-Find data structure (https://fr.wikipedia.org/wiki/Union-Find) is a data structure that
stores a partition of a set into disjoint subsets. Tree height is controlled using union by size.
Use path compression technique. */
template <typename T>
struct UnionFind
{   
    //id is the root of the subset
    //size is to speed up the merging operation
    struct Subset { T id, size; Subset(T _id, T _size) : id(_id), size(_size) {}};   

    std::vector<Subset> subsets;
    T nb_subsets;

    UnionFind(T n) : nb_subsets{n} { 
        subsets.reserve(n);
        for (T i = 0; i < n; ++i)
            subsets.emplace_back(i, 1);
    }


    auto find(T x) {
        auto root = x;
        //path compression as heuristic
        while (root != subsets[root].id) 
            { root = subsets[root].id; } //finding the root element of the subset having itself as a root;

        while (subsets[x].id != root) 
            { 
                auto id = subsets[x].id; //changing the id with the root, and recursively updating also its earlier parent.
                subsets[x].id = root; 
                x = id; 
            }

        return root;
    }

    void merge(T x, T y) {
        auto const i = find(x); 
        auto const j = find(y);

        if (i == j) return; //the two elements belongs to the same subset: no action needed

        //merging the two subsets editing the smaller one for optimization
        if (subsets[i].size < subsets[j].size) 
            { subsets[i].id = j; subsets[j].size += subsets[i].size; }

        else                                   
            { subsets[j].id = i; subsets[i].size += subsets[j].size; }


        --nb_subsets;
        return;
    }

    bool connected(T x, T y) { return find(x) == find(y); }
};

/* A data structure representing a cut of a graph. */
template <typename node_t>
struct GraphCut
{
    std::size_t cut_size;
    UnionFind<node_t> uf; // used to identify the two partitions of nodes
    GraphCut(std::size_t _cut_size, UnionFind<node_t> _uf) : cut_size(_cut_size), uf(_uf) {}

    bool operator<(GraphCut const& other) const { return cut_size < other.cut_size; }

    auto get_partitions() const {
        std::vector<node_t> P, Q;
        P.reserve(uf.subsets[0].size); Q.reserve(std::size(uf.subsets) - std::size(P));
        for (node_t i = 0; i < std::size(uf.subsets); ++i)
            uf.subsets[i].id == uf.subsets[0].id ? P.push_back(i) : Q.push_back(i);
        return std::array{P, Q};
    }
};

/* Karger's contraction algorithm in O(n + mα(n)) using an Union-Find data structure to keep track
of merged vertices. The graph is assumed to be connected and nodes indexed between 0 and n-1. Repeat
this function C(n,2)*log(n) = n*(n-1)/2*log(n) for high probability of obtaining the minimum global
cut. The graph isn't per se modifed, only its vector of edges is shuffled. */
template <typename node_t>
GraphCut<node_t> karger_union_find(EdgesVectorGraph<node_t>* graph)
{
    auto& mt = prng_engine();
    UnionFind uf{ graph->n };

    int start = 0;
    int new_pos=0;
    for (int m = graph->m - 1; uf.nb_subsets != 2; ++start, --m) {

        new_pos = start + std::uniform_int_distribution<>{0, m}(mt);
        std::swap(graph->edges[start], graph->edges[new_pos]);
        uf.merge(graph->edges[start].tail, graph->edges[start].head);

    }

    std::size_t j = 0;    
    for( ; start< (graph->m); start++){
        if(!uf.connected(graph->edges[start].tail, graph->edges[start].head)){
            ++j;
        }
    }


    return GraphCut{j, std::move(uf)};
}


/* Kargen-Stein's contraction recursive algorithm. Instead of using a straightforward recursion, we
keep the intermediate graphs to contract in a stack. Repeat this function log²(n) for high probabili
-ty of obtaining the minimum global cut. */
template <typename node_t>
GraphCut<node_t> karger_stein_union_find(EdgesVectorGraph<node_t> const *input_graph)
{
    /* A data structure to hold an intermediate contracted graph state. The Union-Find structure
    is used to keep track of the merged nodes. */ 
    struct ContractedGraph : EdgesVectorGraph<node_t> { UnionFind<node_t> uf;};

    /* Contracts the given graph until it has nb_vertices vertices. The graph isn't per se modifed,
    only its vector of edges is shuffled. */
    auto contract = [&mt = prng_engine()](ContractedGraph *graph, node_t nb_vertices) {   
        
        UnionFind uf{graph->uf};

        int start = 0;
        int new_pos=0;

        for (int m = graph->m - 1; uf.nb_subsets != nb_vertices; ++start, --m) {
            new_pos = start + std::uniform_int_distribution<>{0, m}(mt);
            std::swap(graph->edges[start], graph->edges[new_pos]);
            uf.merge(graph->edges[start].tail, graph->edges[start].head);
        }

        
        Edge<node_t> *edges = new Edge<node_t> [graph->m - start];
        node_t j = 0;
        for( ; start < (graph->m); start++){
            if(!uf.connected(graph->edges[start].tail, graph->edges[start].head)){
                edges[j] = graph->edges[start];
                ++j;
            }
        }  

        edges = (Edge<node_t> *) realloc( edges, j * sizeof(Edge<node_t>));                   

        return ContractedGraph{ EdgesVectorGraph {nb_vertices, j, std::move(edges)}, std::move(uf)};
    };

    /* Returns the cut represented by an intermediate contracted graph. The given graph is supposed
    to have no self-loops and to have two nodes (components). */
    auto cut = [](ContractedGraph *graph){ return GraphCut{graph->m, std::move(graph->uf)}; };

    double INV_SQRT_2 = 1.0 / std::sqrt(2);
    GraphCut<node_t> best_minimum_cut = {input_graph->n, {{}}};

    std::stack<ContractedGraph, std::vector<ContractedGraph>> graphs;
    graphs.push( ContractedGraph{ *input_graph, (input_graph->n)} );

    bool first = true;
    while (!graphs.empty()) // algorithm's main loop
    {
        auto graph = graphs.top();
        graphs.pop();        
        if (graph.n <= 6) {
            ContractedGraph contr = contract(&graph, 2);
            best_minimum_cut = std::min(best_minimum_cut, cut(&contr));
        } else {
            node_t t = std::ceil(1 + (graph.n) * INV_SQRT_2);
            graphs.push( contract(&graph, t));
            graphs.push( contract(&graph, t));
        }
        
        if(first) first = false;
        else{
            delete[] graph.edges;
        }
    }

    return best_minimum_cut;
}

Overwriting array_impl/karger.hpp


In [ ]:
%%writefile array_impl/instance_reader.hpp
#pragma once

#include <fstream>
#include <sstream>
#include <string>
#include <vector>
#include "karger.hpp"


template <typename node_t>
EdgesVectorGraph<node_t> read_col_instance(std::string_view file) {

    
    std::ifstream instance;
    instance.open(file.data());
    if (!instance) throw std::runtime_error("Such instance doesn't exist.");

    node_t n = 0, m=0;
    Edge<node_t>* edges;
    int i=0;
    for (std::string line; std::getline(instance, line);)
    {
        std::istringstream input(line);      
        
        switch(line[0]) {
            case 'p':
                input.ignore(6);                
                input >> n >> m;
                edges = new Edge<node_t>[m];
                break;
            case 'e':
                input.ignore(2);
                Edge<node_t> edge;
                input >> edge.tail >> edge.head;
                --edge.tail; --edge.head; // instance files starting vertex is 1
                edges[i] = std::move(edge);
                i++;
                break;
            default:
                break;             
        }
    }



   return EdgesVectorGraph<node_t>(n, m, std::move(edges));    
}

Overwriting array_impl/instance_reader.hpp


In [ ]:
%%writefile array_impl/MinCutBM.cpp
#include <algorithm>
#include <iostream>
#include <fstream>
#include <stdlib.h>
#include <cstdint>
#include <functional>
#include <array>

#include <benchmark/benchmark.h>

#include "karger.hpp"
#include "instance_reader.hpp"

static void KargerBM(benchmark::State& state)
{

    using node_t = std::uint32_t;       
            
    for (auto _ : state){
              
      state.PauseTiming();

      int n= state.range(0);
      int m= state.range(1);

      std::string file_name= "/content/toys/"+std::to_string(n) + "_" + std::to_string(m)+".col";

      auto graph = read_col_instance<node_t>(file_name);

    struct MinimumCutAlgorithm {
        std::string name;
        std::function<GraphCut<node_t>(EdgesVectorGraph<node_t>*)> algorithm;
        std::size_t nb_repeat;
        auto operator()(EdgesVectorGraph<node_t>& graph) const { return algorithm(&graph); }
    };

      MinimumCutAlgorithm algorithm = {"Karger",       karger_union_find<node_t>,       static_cast<std::size_t>(0.5 * (graph.n) * (graph.n - 1) * std::log(graph.n))};


      GraphCut<node_t> best_minimum_cut = {graph.n, {{}}};

      state.ResumeTiming();
      for(std::size_t i = algorithm.nb_repeat; i; --i)
        best_minimum_cut = std::min(best_minimum_cut, algorithm(graph)); 


      delete[] graph.edges;
    }

    state.SetComplexityN(state.range(0));
}

static void KargerSteinBM(benchmark::State& state)
{

    using node_t = std::uint32_t;       
            
    for (auto _ : state){
              
      state.PauseTiming();

      int n= state.range(0);
      int m= state.range(1);

      std::string file_name= "/content/toys/"+std::to_string(n) + "_" + std::to_string(m)+".col";

      auto graph = read_col_instance<node_t>(file_name);

    struct MinimumCutAlgorithm {
        std::string name;
        std::function<GraphCut<node_t>(EdgesVectorGraph<node_t>*)> algorithm;
        std::size_t nb_repeat;
        auto operator()(EdgesVectorGraph<node_t>& graph) const { return algorithm(&graph); }
    };

      MinimumCutAlgorithm algorithm = {"Karger-Stein", karger_stein_union_find<node_t>, static_cast<std::size_t>(std::log(graph.n) * std::log(graph.n))};

      GraphCut<node_t> best_minimum_cut{graph.n, {{}}};

      state.ResumeTiming();
      for(std::size_t i = algorithm.nb_repeat; i; --i)
        best_minimum_cut = std::min(best_minimum_cut, algorithm(graph)); 


      delete[] graph.edges;
    }
    state.SetComplexityN(state.range(0));
}

static void CustomArguments(benchmark::internal::Benchmark *b){    
    for(int i = 8; i<=256; i=i*2){
        for(int j = i-1; j <= i*(i-1)/2 ; j=j*2){
            b->Args({i, j});
        }
    }
}


BENCHMARK(KargerBM)
    ->Apply(CustomArguments)
    ->Complexity(benchmark::BigO::oNSquared);

BENCHMARK(KargerSteinBM)
    ->Apply(CustomArguments)
    ->Complexity(benchmark::BigO::oNLogN);


BENCHMARK_MAIN();

Overwriting array_impl/MinCutBM.cpp


In [ ]:
!clang++-10 original_impl/MinCutBM.cpp -O2 -std=c++2a -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o original_impl/kargerBM -g
!clang++-10 array_impl/MinCutBM.cpp -O2 -std=c++2a -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o array_impl/kargerBM -g

In [ ]:
%%writefile toyGenerator.cpp
#include <iostream>
#include <fstream>
#include <stdlib.h>
#include <array>
#include <stdio.h>

#include "original_impl/karger.hpp"


bool createConnection(std::vector<std::vector<bool>> &graph, UnionFind<int> &dfs){ 

    int max=0, min = 0, curr=0, most=0, least=0;    

    curr = dfs.subsets[dfs.find(0)].size;
    max=curr;
    min=curr;

    for(int i=1; i<dfs.subsets.size(); i++){

        curr = dfs.subsets[ dfs.find(i) ].size;

        if(curr > max){
            max = curr;
            most=i;
        }else if(curr < min){
            min = curr;
            least = i;
        }
    }
    //the case in which all the subsets have the same size
    if(dfs.connected(most, least))
    {
      for(int i=0; i<dfs.subsets.size(); i++){
          if(! dfs.connected(most, i)){
            least=i; //we take the first element we found of a set different from that of "most"
            break;
          } 
      }
    }

    graph.at(most).at(least)=1;
    dfs.merge(most, least);
    return true;
}


void generate_graph(int n, int m)
{   

  std::random_device rd;  //Will be used to obtain a seed for the random number engine
  std::mt19937 gen(rd()); //Standard mersenne_twister_engine seeded with rd()
  std::uniform_int_distribution<> distrib(0, n-1);
  
  std::vector<std::vector<bool>> graph (n, std::vector<bool>(n, 0));
  std::vector<bool> visited(n, 0);
  UnionFind<int> dfs(n);


  int index1, index2, checks=0, total_edges = 0;

  /*
  ----------------------------------------------------CREATING THE GRAPH----------------------------------------------------------
      In order to create where no node is isolated, we first connect each node to another picked randomly.
      We are sure that all the nodes are connected when at least n-1 edges are placed, which is the minimum required number. 
      Then we create,  randomly as well, other edges.
  */
  while (total_edges < n-1)
  {
      index1 = distrib(gen);
      index2 = distrib(gen);

      if( dfs.connected(index1, index2) )
      { 
          if(checks >= n){

              createConnection(graph, dfs);                
              
              checks=0;
              ++total_edges;
          }
          else  ++checks;

      }
      else{   
          checks = 0;
          graph.at(index1).at(index2) = 1;
          dfs.merge(index1, index2);
          ++total_edges;
      }        
  }
  
  total_edges=0;
  while (total_edges < m-n+1)
  {
      index1 = distrib(gen);
      index2 = distrib(gen);

      if( !(index1 == index2 || graph.at(index1).at(index2)) ){
          graph.at(index1).at(index2) = 1;
          ++total_edges;
      }        
  } 
  
  std::ofstream toyFile;
  std::string toyname = "toys/"+std::to_string(n) + "_" + std::to_string(m)+".col";
  toyFile.open(toyname);
  toyFile << "p edge "<<n<<" "<<m; 

  for (int i = 0; i < n; ++i)
  {
      for(int j=0; j < n; ++j)
      {
          if(graph.at(i).at(j))
          {
              toyFile << "\ne "<<i+1<<" "<<j+1;
          }
      }     
  }
}

int main(int argc, char* argv[])
{
  for(int i = 8; i<=256; i=i*2)
  {
    for(int j = i-1; j <= i*(i-1)/2 ; j=j*2)
    {        
      generate_graph(i, j);
    }
  }
  
}

Overwriting toyGenerator.cpp


In [ ]:
! rm -rf toys
!clang++-10 toyGenerator.cpp -O2 -std=c++2a -o toyGenerator -g

! mkdir toys
!./toyGenerator

In [ ]:
! benchmark/tools/compare.py benchmarks /content/original_impl/kargerBM /content/array_impl/kargerBM

RUNNING: /content/original_impl/kargerBM --benchmark_out=/tmp/tmpsw764rre
2021-10-18T18:56:19+00:00
Running /content/original_impl/kargerBM
Run on (2 X 2200.22 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 0.72, 0.78, 0.41
------------------------------------------------------------------
Benchmark                        Time             CPU   Iterations
------------------------------------------------------------------
KargerBM/8/7                 20062 ns        19805 ns        35021
KargerBM/8/14                29978 ns        29727 ns        23486
KargerBM/8/28                39892 ns        39313 ns        17628
KargerBM/16/15              250961 ns       249354 ns         2844
KargerBM/16/30              366466 ns       365595 ns         1910
KargerBM/16/60              457609 ns       456501 ns         1519
KargerBM/16/120             624828 ns       622023 ns         1126
KargerBM/32